In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.io as pio
train = pd.read_csv('drive/MyDrive/mfk/train.csv')
test = pd.read_csv('drive/MyDrive/mfk/test.csv')


In [ ]:
train['mode'] = 'train'
test['mode'] = 'test'

df = pd.concat([train, test])
df

,id,date,city_name,store_id,category_id,product_id,price,weather_desc,humidity,temperature,pressure,sales,mode
0,1,2021-07-29,Москва,1,1,1,4.79,"переменная облачность, небольшой дождь",61.9375,23.1875,741.0000,26.0,train
1,2,2021-07-30,Москва,1,1,1,4.79,"переменная облачность, небольшой дождь",70.2500,22.1875,740.3125,37.0,train
2,3,2021-07-31,Москва,1,1,1,4.79,переменная облачность,52.6250,21.8125,741.6250,25.0,train
3,4,2021-08-01,Москва,1,1,1,4.79,"облачно, небольшой дождь",87.4375,20.0625,743.3125,26.0,train
4,5,2021-08-02,Москва,1,1,1,4.79,переменная облачность,66.1875,23.4375,739.6250,22.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24831,691508,2022-02-16,Воронеж,164,1,35,2.48,переменная облачность,91.8125,-2.5625,751.0000,NaN,test
24832,691509,2022-02-17,Воронеж,164,1,35,2.48,"облачно, небольшой дождь",93.5000,2.0625,740.0000,NaN,test
24833,691510,2022-02-18,Воронеж,164,1,35,2.48,облачно,96.5625,2.0625,736.0000,NaN,test
24834,691511,2022-02-19,Воронеж,164,1,35,2.48,переменная облачность,89.9375,2.7500,743.0000,NaN,test


In [ ]:
df = df.sort_values(['store_id', 'product_id', 'date']).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 691512 entries, 0 to 691511
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            691512 non-null  int64  
 1   date          691512 non-null  object 
 2   city_name     691512 non-null  object 
 3   store_id      691512 non-null  int64  
 4   category_id   691512 non-null  int64  
 5   product_id    691512 non-null  int64  
 6   price         691512 non-null  float64
 7   weather_desc  691512 non-null  object 
 8   humidity      691512 non-null  float64
 9   temperature   691512 non-null  float64
 10  pressure      691512 non-null  float64
 11  sales         666676 non-null  float64
 12  mode          691512 non-null  object 
dtypes: float64(5), int64(4), object(4)
memory usage: 68.6+ MB


In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit(df.city_name)
df['city_le']=le.transform(df.city_name)
le.fit(df.weather_desc)
df['weather_le']=le.transform(df.weather_desc)
df = pd.concat((df, pd.get_dummies(df.weather_le, prefix='wether')), axis=1)
df = pd.concat((df, pd.get_dummies(df.city_le, prefix='city')), axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 691512 entries, 0 to 691511
Data columns (total 41 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            691512 non-null  int64  
 1   date          691512 non-null  object 
 2   city_name     691512 non-null  object 
 3   store_id      691512 non-null  int64  
 4   category_id   691512 non-null  int64  
 5   product_id    691512 non-null  int64  
 6   price         691512 non-null  float64
 7   weather_desc  691512 non-null  object 
 8   humidity      691512 non-null  float64
 9   temperature   691512 non-null  float64
 10  pressure      691512 non-null  float64
 11  sales         666676 non-null  float64
 12  mode          691512 non-null  object 
 13  city_le       691512 non-null  int64  
 14  weather_le    691512 non-null  int64  
 15  wether_0      691512 non-null  uint8  
 16  wether_1      691512 non-null  uint8  
 17  wether_2      691512 non-null  uint8  
 18  weth

In [ ]:
df.date = pd.to_datetime(df.date)
df['weekday'] = df.date.dt.weekday
df['weekend'] = df['weekday'].isin([5, 6]).astype(int)
group = df.groupby(['store_id', 'product_id'])
for i in range(7, 21):
    df[f'lag_day_{i}'] = group['sales'].shift(i)
#df[(df['product_id'] == 1) & (df['store_id'] == 1)].iloc[-20:][['date', 'lag_day_11', 'lag_day_7', 'sales', 'mode']]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 691512 entries, 0 to 691511
Data columns (total 57 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   id            691512 non-null  int64         
 1   date          691512 non-null  datetime64[ns]
 2   city_name     691512 non-null  object        
 3   store_id      691512 non-null  int64         
 4   category_id   691512 non-null  int64         
 5   product_id    691512 non-null  int64         
 6   price         691512 non-null  float64       
 7   weather_desc  691512 non-null  object        
 8   humidity      691512 non-null  float64       
 9   temperature   691512 non-null  float64       
 10  pressure      691512 non-null  float64       
 11  sales         666676 non-null  float64       
 12  mode          691512 non-null  object        
 13  city_le       691512 non-null  int64         
 14  weather_le    691512 non-null  int64         
 15  wether_0      691

In [ ]:
cat_features = ['store_id', 'category_id', 'product_id', 'weekday','weekend', 'city_0',
 'city_1',
 'city_2',
 'city_3',
 'city_4',
 'city_5',
 'city_6',
 'city_7',
 'city_8', 'city_9', 'wether_0',
 'wether_1',
 'wether_2',
 'wether_3',
 'wether_4',
 'wether_5',
 'wether_6',
 'wether_7',
 'wether_8',
 'wether_9',
 'wether_10',
 'wether_11',
 'wether_12',
 'wether_13',
 'wether_14',
 'wether_15']
num_features =['lag_day_7','lag_day_8','lag_day_9','lag_day_10','lag_day_11','lag_day_12','lag_day_13','lag_day_14','lag_day_15','lag_day_16','lag_day_17','lag_day_18','lag_day_19','lag_day_20',
               'price','temperature','pressure','humidity']
train_df = df[(df['mode'] == 'train')].copy()
train_df=train_df.sort_values(by='lag_day_20')
train_df.info()
train_df=train_df.iloc[:595716]



<class 'pandas.core.frame.DataFrame'>
Int64Index: 666676 entries, 463281 to 691480
Data columns (total 57 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   id            666676 non-null  int64         
 1   date          666676 non-null  datetime64[ns]
 2   city_name     666676 non-null  object        
 3   store_id      666676 non-null  int64         
 4   category_id   666676 non-null  int64         
 5   product_id    666676 non-null  int64         
 6   price         666676 non-null  float64       
 7   weather_desc  666676 non-null  object        
 8   humidity      666676 non-null  float64       
 9   temperature   666676 non-null  float64       
 10  pressure      666676 non-null  float64       
 11  sales         666676 non-null  float64       
 12  mode          666676 non-null  object        
 13  city_le       666676 non-null  int64         
 14  weather_le    666676 non-null  int64         
 15  wether_0    

In [ ]:
train_df['lag_day_20']

463281      0.0
644059      0.0
549829      0.0
549827      0.0
644065      0.0
          ...  
283538    253.0
88411     264.0
88418     268.0
88419     269.0
605093    270.0
Name: lag_day_20, Length: 595716, dtype: float64

In [ ]:
test_df = df[(df['mode'] == 'test')].copy()

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24836 entries, 200 to 691511
Data columns (total 57 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            24836 non-null  int64         
 1   date          24836 non-null  datetime64[ns]
 2   city_name     24836 non-null  object        
 3   store_id      24836 non-null  int64         
 4   category_id   24836 non-null  int64         
 5   product_id    24836 non-null  int64         
 6   price         24836 non-null  float64       
 7   weather_desc  24836 non-null  object        
 8   humidity      24836 non-null  float64       
 9   temperature   24836 non-null  float64       
 10  pressure      24836 non-null  float64       
 11  sales         0 non-null      float64       
 12  mode          24836 non-null  object        
 13  city_le       24836 non-null  int64         
 14  weather_le    24836 non-null  int64         
 15  wether_0      24836 non-null  uin

In [ ]:
test_df = df[df['mode'] == 'test'].copy()
from sklearn.preprocessing import StandardScaler
y_train = train_df.sales
y_test = test_df.sales

scaler = StandardScaler()

X_train_cat = train_df[cat_features].values
X_test_cat = test_df[cat_features].values


X_train_num = train_df[num_features].copy()
X_test_num = test_df[num_features].copy()

scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train_num)
X_test_num = scaler.transform(X_test_num)


X_train = pd.DataFrame(np.hstack((X_train_num, X_train_cat)), columns=num_features+cat_features)
X_test = pd.DataFrame(np.hstack((X_test_num, X_test_cat)), columns=num_features+cat_features)

In [ ]:
y_train

463281     48.0
644059      2.0
549829      2.0
549827      2.0
644065      1.0
          ...  
283538    133.0
88411     211.0
88418     186.0
88419     231.0
605093     72.0
Name: sales, Length: 595716, dtype: float64

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model=GradientBoostingRegressor(n_estimators=200,
          max_depth=15,
          learning_rate=0.05,
          min_samples_leaf=16,
          min_samples_split=16, verbose = True)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
output = pd.DataFrame({'id': test.id, 'prediction': predictions})

      Iter       Train Loss   Remaining Time 
         1         212.8523           32.46m
         2         195.2435           30.96m
         3         179.2928           30.34m
         4         164.8354           29.95m
         5         151.7300           29.59m
         6         139.8521           29.33m
         7         129.0926           29.11m
         8         119.3240           28.89m
         9         110.4571           28.76m
        10         102.3973           29.44m
        20          52.9213           27.29m
        30          33.0465           25.72m
        40          24.4047           24.12m
        50          20.1199           22.57m
        60          17.6347           20.99m
        70          16.0960           19.43m
        80          14.8967           17.87m
        90          14.0127           16.32m
       100          13.3429           14.78m
       200          10.1657            0.00s


In [ ]:
output.to_csv('prediction.csv', index=False)
output

,id,prediction
0,666677,24.956883
1,666678,23.613072
2,666679,27.060249
3,666680,28.149772
4,666681,28.571261
...,...,...
24831,691508,10.909246
24832,691509,13.290373
24833,691510,13.360464
24834,691511,16.389692


In [ ]:
import tensorflow as tf
from tensorflow import keras
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100, input_dim=3, activation='sigmoid'))
model.add(tf.keras.layers.ReLU())
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(50, activation='sigmoid'))
model.add(tf.keras.layers.ReLU())
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(25, activation='softmax'))

#IMPORTANT PART
model.add(tf.keras.layers.Dense(1, activation='linear'))

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                 optimizer=tf.keras.optimizers.Adam(),
                 metrics=['mse'])

In [ ]:
from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor = "val_loss" ,
mode = "min" , patience = 5 ,
restore_best_weights = True )
history = model.fit(X_train,y_train,verbose = 1, validation_split = 0.1, callbacks = [earlystopping], epochs = 50)

Epoch 1/50
16755/16755 [==============================] - 45s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/50
16755/16755 [==============================] - 45s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/50
16755/16755 [==============================] - 47s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/50
16755/16755 [==============================] - 45s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/50
16755/16755 [==============================] - 45s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan


In [ ]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
models = []
#models.append(('LR', LinearRegression()))
#models.append(('RFR',RandomForestRegressor(n_estimators=100, max_depth=5, random_state=1)))
models.append(('GBR', GradientBoostingRegressor(n_estimators=200,
          max_depth=12,
          criterion='mse',
          learning_rate=0.03,
          min_samples_leaf=16,
          min_samples_split=16)))
results = []
model_names = []
for name, model in models:
    tscv = TimeSeriesSplit(n_splits=4)
    cv_results = cross_val_score(model, X_train, y_train, cv=tscv, scoring='r2')
    results.append(cv_results)
    model_names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))